In [32]:
import pandas as pd
import numpy as np
import re

In [33]:
rdf = pd.read_csv("./data/Aged-Care-Homes-June-2018.csv")

omit_col = ['ALT_NAME', 'PREVIOUS_NAME', 'LOCAL_NAME',
       'COMMGOVT_SUBSIDISED', 'BUS_UNIT_NO', 'BUS_ST_ADDRESS', 'BUS_SUBURB', 'BUS_PCODE', 'BUS_STATE',
       'MAIN_EMAIL_ADDR', 'SERVICE_TYPE', 'ABN', 'APPROVED_PROVIDER', 'ACCREDITATION',
       'ACCREDITATION_PERIOD', 'CERTIFICATION', 'NOTICE_OF_SANCTION',
       'NOTICE_OF_NON_COMPLIANCE', 'AVAILABILITY', 'ROOM_TITLE', 'ROOM_NAME', 'ROOM_TYPE', 'MAX_ROOM_OCCUPANCY',
       'NUM_OF_ROOM_TYPES', 'MAX_RAD', 'MAX_DAP', 'PAYMENT_OPTIONS',
       'PAYMENT_EXAMPLE', 'ROOM_DESC', 'ROOM_SIZE', 'COMMON_AREA_DESC',
       'ADDITIONAL_CARE_INCL', 'ADDITIONAL_CARE_COST',
       'EXTRA_SERVICE', 'EXTRA_SERVICE_FEE_AMOUNT', 'SPECIFIC_FEATURES']

rdf.drop(labels=omit_col, axis=1, inplace=True)
rdf.drop_duplicates(inplace=True)
rdf.head()

,SERVICE_NAME,DESCRIPTION,STREET_UNIT_NO,STREET_ST_ADDRESS,STREET_SUBURB,STREET_PCODE,STREET_STATE,MAIN_PH_NUM,MAIN_FAX_NUM,WEBSITE,PARTICULAR_NEED_SERVICES
0,Juniper Cygnet,"Juniper Cygnet located on, the Juniper Rowetho...",NaN,4-10 HAYMAN Road,BENTLEY,6102.0,WA,+611300313000,+610892400329,www.juniper.org.au,Focus on socially and financially disadvantage...
3,Blue Care Labrador Aged Care Facility,Aged Care Residential Facility,NaN,83 MUIR Street,LABRADOR,4215.0,QLD,+611800838929,NaN,http://www.bluecare.org.au,Focus on socially and financially disadvantage...
5,RSL Care Bolton Point - Macquarie Shores,NaN,NaN,12 The Ridgeway,BOLTON POINT,2283.0,NSW,+610249503933,+610249593498,http://www.rslcare.com.au/bolton-point,NaN
6,RSL Care Bolton Point - Macquarie Shores,NaN,NaN,12 The Ridgeway,BOLTON POINT,2283.0,NSW,+610249503933,+610249593498,http://www.rslcare.com.au/bolton-point,"Caters for cultural, spiritual or ethical food..."
9,Ark Health Care Bankstown,NaN,NaN,1A HIXSON Street,BANKSTOWN,2200.0,NSW,+610297919609,+610297915609,http://www.arkhc.com,Focus on socially and financially disadvantage...


In [34]:
def special_serv():
    col_names = ["sfd", "dem", "atsi", "terminal", "veteran", "cald", "lgbti"]
    search_term = ["financially disadvantaged", "dementia", "ATSI", "terminal illness", "veterans", "CALD backgrounds", "LGBTI"]
    for ix, col in enumerate(col_names):
        pattern = re.compile(search_term[ix])
        bool_list = [False] * rdf.shape[0]
        for rdf_ix, service_string in enumerate(rdf["PARTICULAR_NEED_SERVICES"]):
            try:
                if pattern.search(service_string):
                    bool_list[rdf_ix] = True
            except TypeError:
                pass
        rdf[col] = bool_list
    rdf.drop("PARTICULAR_NEED_SERVICES", axis=1, inplace=True)
special_serv()
rdf.shape

(5070, 17)

In [39]:
def remove_illegals(desc):
    try:
        return re.sub(pattern=r'\’|\é',repl='',string=desc)
    except TypeError:
        return ""

rdf["DESCRIPTION"] = rdf["DESCRIPTION"].apply(remove_illegals)

rdf.drop_duplicates(subset=["SERVICE_NAME"], inplace=True)
rdf.shape

(2863, 17)

In [40]:
rdf.to_csv("./data/clean/residential_basic.csv")